# Summary

The goal of this notebook is to understand the efficacy of fitting SAASGP to individual principal components.

Currently, we implement PCA-GP through an OutcomeTransform(), which performs the outcome transform and fits batch single-task GPs to the principal components.

For problems with relatively high input dimensionality (e.g., lunar lander problem requires 12-dimensional input), it might be reasonable to impose some sparsity on the inputs. This can be done through fitting SAASGPs to the principal components.

However, SAASGP does not currently support batch / multi-output fitting as the single-task GPs do. So making this change would be a lot of work.

In this notebook I would like to understand how much value SAASGP can provide. Will take the following steps:
- Use lunar lander as example, generate a dataset of input and outputs
- Option 1: fit GP using PCAOutcomeTransform
- Option 2: manually do PCA transform, fit individual saasgp models to each of the PCs
- Diagnostic 1: Compare the outcome model MSE on a separate test set for option 1 vs option 2
- Diagnostic 2: Are different input dimensions selected for different PCs? Our hypothesis is that they are different. Is this really the case?

In [35]:
%load_ext autoreload
%autoreload 2

import itertools
import pickle
import re
import warnings
from collections import defaultdict
from dataclasses import asdict, dataclass
from typing import Dict, List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns

warnings.filterwarnings("ignore")

import copy
import random
import time
from collections import defaultdict
from typing import Any, Dict, List, NamedTuple
import gpytorch
import numpy as np
import torch
import sys
sys.path.append('..')


from low_rank_BOPE.src.lunar_lander import LunarLander
from low_rank_BOPE.src.pref_learning_helpers import (
    check_outcome_model_fit,
    check_pref_model_fit,
    find_max_posterior_mean,
    fit_outcome_model,
    fit_pref_model,
    gen_exp_cand,
    generate_random_exp_data,
    generate_random_pref_data,
    run_pref_learn
)
from low_rank_BOPE.src.transforms import (
    generate_random_projection,
    InputCenter,
    LinearProjectionInputTransform,
    LinearProjectionOutcomeTransform,
    PCAInputTransform,
    PCAOutcomeTransform,
    SubsetOutcomeTransform,
)
from low_rank_BOPE.src.models import make_modified_kernel, MultitaskGPModel
from low_rank_BOPE.src.diagnostics import (
    empirical_max_outcome_error,
    empirical_max_util_error,
    mc_max_outcome_error,
    mc_max_util_error,
)
from low_rank_BOPE.src.saasgp_utils import (
    SaasPriorHelper,
    add_saas_prior,
    _get_map_saas_model,
    get_fitted_map_saas_model,
    get_and_fit_map_saas_model
)

from botorch.acquisition.objective import GenericMCObjective, LearnedObjective
from botorch.fit import fit_gpytorch_mll, fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_scipy
from botorch.models.multitask import KroneckerMultiTaskGP
from botorch.models.transforms.input import (
    ChainedInputTransform,
    FilterFeatures,
    Normalize,
)

from botorch.models.transforms.outcome import ChainedOutcomeTransform, Standardize
# from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.test_functions.base import MultiObjectiveTestProblem

from gpytorch.kernels import LCMKernel, MaternKernel
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from gpytorch.priors import GammaPrior
from gpytorch.priors.lkj_prior import LKJCovariancePrior

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
config = {
    "initial_experimentation_batch": 128,
    "n_check_post_mean": 13,
    "every_n_comps": 3,
    "outcome_dim": 20
}

tkwargs = {'dtype': torch.double}

In [95]:
problem = LunarLander(num_envs=20)
X, Y = generate_random_exp_data(problem, config["initial_experimentation_batch"], batch_eval = False)


In [96]:
X.shape, Y.shape

(torch.Size([128, 12]), torch.Size([128, 20]))

In [97]:
# create transformation that does standardization and PCA
std_pca_outcome_transform = ChainedOutcomeTransform(
        **{
            "standardize": Standardize(
                config["outcome_dim"],
                min_stdv=120,  # TODO: setting 120 means not standardizing
            ),
            # "pca": PCAOutcomeTransform(num_axes=config["lin_proj_latent_dim"]),
            "pca": PCAOutcomeTransform(
                variance_explained_threshold=0.9
            ),
        }
    )

In [98]:
outcome_model_pca = fit_outcome_model(
    X,
    Y,
    outcome_transform=std_pca_outcome_transform,
)

In [99]:
# How many axes out of 20 explain >90% of variance
print('explained variance: ', outcome_model_pca.outcome_transform['pca'].PCA_explained_variance)
print('number of axes: ', outcome_model_pca.outcome_transform['pca'].num_axes.item())

num_axes = outcome_model_pca.outcome_transform['pca'].num_axes.item()
axes_learned = outcome_model_pca.outcome_transform['pca'].axes_learned

explained variance:  tensor(0.9162, dtype=torch.float64)
number of axes:  8


In [100]:
# Qing's code for fitting individual saasgp's to the PCs

Y_transformed = std_pca_outcome_transform(Y)[0]
Xs = [X for _ in range(num_axes)]
Ys = [Y_transformed[:, [i]] for i in range(num_axes)]
Yvars = [
    torch.full(Y_transformed[:, [i]].size(), torch.nan, **tkwargs)
    for i in range(num_axes)
]


In [101]:
outcome_model_pca_saas = get_and_fit_map_saas_model(
    Xs = Xs,
    Ys = Ys,
    Yvars = Yvars,
    task_features = [],
    fidelity_features = [],
    metric_names = []
)

## next check posterior fitting error on outcomes (map back)

In [102]:
test_X = torch.rand((30, 12))
# evaluate the outcomes sequentially because the lunar lander does not support batch evaluation
test_Y_list = []
for idx in range(len(test_X)):
    # print('X[idx]', idx, X[idx])
    # print('problem(X[idx])', problem(X[idx]))
    y = problem(test_X[idx]).detach()
    test_Y_list.append(y)
test_Y = torch.stack(test_Y_list).squeeze(1)

outcome_post_mean_pca = outcome_model_pca.posterior(test_X).mean
outcome_post_mean_pca_saas = torch.matmul(outcome_model_pca_saas.posterior(test_X).mixture_mean, axes_learned)

In [103]:
# sum of squared errors for the PCA model using OutcomeTransform
torch.linalg.norm(outcome_post_mean_pca - test_Y)

tensor(1568.9202, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>)

In [104]:
# sum of squared errors for the PCA model using individual SAASGPs
torch.linalg.norm(outcome_post_mean_pca_saas - test_Y)

tensor(7997.6420, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>)

## (outdated) Also check what input dimensions are selected for each of the saasgps

Important input dimensions found by the MAP-SAAS model:

- PC 0: 4, 5, 11
- PC 1: 5, 7, 10
- PC 2: not very consistent across 4 different hyperparameters

In [65]:
outcome_model_pca_saas.models[2].covar_module.base_kernel.lengthscale


tensor([[[1.0000e+04, 3.8202e+01, 9.9994e+03, 1.0000e+04, 1.0000e+04,
          9.9949e+03, 1.0000e+04, 1.0000e+04, 1.0000e-02, 1.0000e+04,
          1.0000e+04, 1.0000e-02]],

        [[1.0000e+04, 1.0000e+04, 2.1990e-01, 1.0000e+04, 1.0000e+04,
          9.7685e+03, 1.0000e+04, 1.0000e+04, 1.0000e+04, 1.0000e+04,
          9.8017e+03, 3.8990e-02]],

        [[1.0000e+04, 1.0000e-02, 1.0000e-02, 1.0000e+04, 1.0000e+04,
          9.7128e+03, 1.0000e+04, 1.0000e+04, 1.0000e-02, 1.0000e+04,
          9.9689e+03, 1.0000e+04]],

        [[1.0000e+04, 2.0127e+00, 9.9999e+03, 1.0000e+04, 1.0000e+04,
          9.9977e+03, 1.0000e+04, 1.0000e+04, 1.0000e-02, 1.0000e+04,
          1.0000e+04, 1.0000e-02]]], dtype=torch.float64,
       grad_fn=<SoftplusBackward0>)